光学字符识别（Optical Character Recognition, OCR）是指对文本材料的图像文件进行分析识别处理，以获取文字和版本信息的过程。也就是说将图象中的文字进行识别，并返回文本形式的内容。例如（该预测效果基于PaddleHub一键OCR中文识别效果展示）：

In [ ]:
#由于PaddleHub升级比较快，建议大家直接升级到最新版本的PaddleHub，无需指定版本升级
!pip install paddlehub --upgrade -i https://pypi.tuna.tsinghua.edu.cn/simple
#该Module依赖于第三方库shapely、pyclipper，使用该Module之前，请先安装shapely、pyclipper
!pip install shapely -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install pyclipper -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
# 待预测图片
test_img_path = ["./advertisement.jpg", "./pics.jpg", "./identity_card.jpg", "./express.jpg", "./railway_ticket.jpg"]

# 展示其中广告信息图片
img1 = mpimg.imread(test_img_path[0])
plt.figure(figsize=(10,10))
plt.imshow(img1)
plt.axis('off')
plt.show()

PaddleHub提供了以下文字识别模型：

移动端的超轻量模型：仅有8.6M，chinese_ocr_db_crnn_mobile。

服务器端的精度更高模型：识别精度更高，chinese_ocr_db_crnn_server。

识别文字算法均采用CRNN（Convolutional Recurrent Neural Network）即卷积递归神经网络。其是DCNN和RNN的组合，专门用于识别图像中的序列式对象。与CTC loss配合使用，进行文字识别，可以直接从文本词级或行级的标注中学习，不需要详细的字符级的标注。该Module支持直接预测。 移动端与服务器端主要在于骨干网络的差异性，移动端采用MobileNetV3，服务器端采用ResNet50_vd。

In [ ]:
import paddlehub as hub

# 加载移动端预训练模型
ocr = hub.Module(name="chinese_ocr_db_crnn_mobile")
# 服务端可以加载大模型，效果更好
# ocr = hub.Module(name="chinese_ocr_db_crnn_server")

PaddleHub对于支持一键预测的module，可以调用module的相应预测API，完成预测功能。

In [ ]:
import cv2

# 读取测试文件夹test.txt中的照片路径
np_images =[cv2.imread(image_path) for image_path in test_img_path]

results = ocr.recognize_text(
                    images=np_images,         # 图片数据，ndarray.shape 为 [H, W, C]，BGR格式；
                    use_gpu=False,            # 是否使用 GPU；若使用GPU，请先设置CUDA_VISIBLE_DEVICES环境变量
                    output_dir='ocr_result',  # 图片的保存路径，默认设为 ocr_result；
                    visualization=True,       # 是否将识别结果保存为图片文件；
                    box_thresh=0.5,           # 检测文本框置信度的阈值；
                    text_thresh=0.5)          # 识别中文文本置信度的阈值；

for result in results:
    data = result['data']
    save_path = result['save_path']
    for infomation in data:
        print('text: ', infomation['text'], '\nconfidence: ', infomation['confidence'], '\ntext_box_position: ', infomation['text_box_position'])

借助 PaddleHub，服务器端的部署也非常简单，直接用一条命令行在服务器启动文字识别OCR模型：

`hub serving start -m chinese_ocr_db_crnn_mobile -p 8866`
是的，在服务器端这就完全没问题了。相比手动配置各种参数或者调用各种框架，PaddleHub 部署服务器实在是太好用了。

只要在服务器端完成部署，剩下在客户端调用就不会有多大问题了。如下百度展示了调用服务器做推断的示例：制定要预测的图像列表、发出推断请求、返回并保存推断结果。

In [ ]:
# coding: utf8
import requests
import json
import cv2
import base64

def cv2_to_base64(image):
    data = cv2.imencode('.jpg', image)[1]
    return base64.b64encode(data.tostring()).decode('utf8')

# 发送HTTP请求
data = {'images':[cv2_to_base64(cv2.imread("/PATH/TO/IMAGE"))]}
headers = {"Content-type": "application/json"}
url = "http://127.0.0.1:8866/predict/chinese_ocr_db_crnn_mobile"
r = requests.post(url=url, headers=headers, data=json.dumps(data))

# 打印预测结果
print(r.json()["results"])